<h1>Market Income Poverty<h1>
    <h2>In this script we try to replicate the market income poverty ratio of the National Insurance Institute.<h2>

Importing the required libraries.

In [10]:
import pandas as pd
import numpy as np

A function that calculates the weighted median of a given series of values and weights.

In [11]:
def weighted_median(data, weights, interpolate = True):
    """
    A function that calculates the weighted median of a given series of values 
    by using a series of weights.
    
    Parameters
    ----------
    data : Iterable
        The data which the function calculates the median for.
    weights : Iterable
        The weights the function uses to calculate an weighted median.

    Returns
    -------
    numpy.float64
        The function return the weighted median.
        
    Required libraries
    ---------
    Numpy (import as np).
    """
    #Forcing the data to a numpy array.
    data = np.array(data)
    weights = np.array(weights)
    
    #Sorting the data and the weights.
    ind_sorted = np.argsort(data)
    sorted_data = data[ind_sorted]
    sorted_weights = weights[ind_sorted]
   
    #Calculating the cumulative sum of the weights.
    sn = np.cumsum(sorted_weights)
    
    #Calculating the threshold.
    threshold = sorted_weights.sum()/2
    
    #Returning the first value that equals or larger than the threshold.
    if interpolate == False:
        return sorted_data[sn >= threshold][0]
    
    #Else interpolating the median and returning it.
    else:
        return np.interp(0.5, (sn - 0.5 * sorted_weights) / np.sum(sorted_weights), sorted_data)
    

Replace the k-data address here.

In [12]:
address = r'C:\Users\dtsj8\Google Drive (tsadeh@kohelet.org.il)\k_data\CBS Households Expenditures Survey\famexp_'

The file names of the relevant surveys.

In [13]:
file_names = ['H20171021datamb.csv','H20181021datamb.csv']
years = [2017,2018]

Creating an empty DataFrame to contain the results.

In [14]:
results = pd.DataFrame()

The main loop, with notes detailing each line of code.

In [15]:
for year, file in zip(years, file_names):
    #Reading the relevant file and saving it to a DataFrame.
    df = pd.read_csv(address + str(year) +'\\'+ file)
    
    #Calculating income per standard person by dividing the net income of the household by the number of standard person in each household.
    df['income_per_standardised_capita'] = df['net'] / df['nefeshstandartit']
    
    #Calculating the market income of households by summing the income from work, capital and pensions in each household. 
    #Gives the same results as substracting the income from allowances and assistance from the gross income of the household.
    df['market_income'] = df['i11'] + df['i12'] + df['i13']
    
    #Calculating the market income per standard person of households by dividng the market income of the household by the number of standard persons in each household.
    df['market_income_per_standardised_capita'] = df['market_income'] / df['nefeshstandartit']
    
    #Calculating household income without work income.
    df['no_work_income'] = df['i12'] + df['i13'] + df['i14']
    
    #Calculating household income without work income per standard person, by dividing the no work income by the number of standard persons in each household.
    df['no_work_income_per_standardised_capita'] = df['no_work_income'] / df['nefeshstandartit']
    
    #Calculating the total persons each household represent in the general population, by multiplying the number of persons in each household by its weight.
    df['total_persons'] = df['nefashot'] * df['weight']
    
    #Calculating the poverty line as half the median net income per standard person.
    poverty_line = weighted_median(df['income_per_standardised_capita'], df['weight'], interpolate = False) / 2
    
    #Calculating the standard poverty ratio of households, by dividing the number of households that their net income per standard person is below the poverty line, by the total number of households.
    results.loc[year, 'standard_poverty_households'] =  df[df['income_per_standardised_capita'] <= poverty_line]['weight'].sum() / df['weight'].sum()
    
    #Calculating the market income poverty ratio of households, by dividing the number of households that their market income per standard person is below the poverty line, by the total number of households.
    results.loc[year, 'market_income_below_line_households'] = df[df['market_income_per_standardised_capita'] <= poverty_line]['weight'].sum() / df['weight'].sum()
    
    #Calculating the no work income poverty ratio of households, by dividing the number of households that their income without work per standard person is below the poverty line, by the total number of households.
    results.loc[year, 'no_work_below_line_households'] = df[df['no_work_income_per_standardised_capita'] <= poverty_line]['weight'].sum() / df['weight'].sum()
    
    #The same calculations but with persons and not households.
    results.loc[year, 'standard_poverty_persons'] =  df[df['income_per_standardised_capita'] <= poverty_line]['total_persons'].sum() / df['total_persons'].sum()
    results.loc[year, 'market_income_below_line_persons'] = df[df['market_income_per_standardised_capita'] <= poverty_line]['total_persons'].sum() / df['total_persons'].sum()
    results.loc[year, 'no_work_below_line_persons'] = df[df['no_work_income_per_standardised_capita'] <= poverty_line]['total_persons'].sum() / df['total_persons'].sum()



The address to export the results to. Don't erase the 'r'. 

In [16]:
address_results = r'C:\Users\dtsj8\OneDrive\Documents\Work'

Exporting the results.

In [17]:
results.to_csv(address_results + '\\results_poverty_market_income.csv')

The NII results in 2017 are: <br>
28.4% market income poverty of households. Ours is 27.98%. <br>
28.0% market income poverty of individuals. Ours is 27.76%. 

The NII results in 2018 are: <br>
27.8% market income poverty of households. Ours is 27.28%. <br>
28.0% market income poverty of individuals. Ours is 27.57%. <br>

<a href="https://www.btl.gov.il/Publications/oni_report/Documents/oni2018.pdf">Source</a>

In 2018, NII results for market income per household are 18,768 NIS.
According to the CBS, it's 18,857 NIS (21,063 - 2,206).

<a href="https://www.cbs.gov.il/he/publications/doclib/2020/1802_households_2018/t02.pdf">Source</a>